In [49]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from gensim.models import word2vec
import itertools
from itertools import product
from itertools import combinations 
from tqdm import tqdm

In [2]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [3]:
# Read recipe inputs
model_folder_path = dataiku.Folder("m9JZdV7b").get_path()
model_path = model_folder_path + "/word2vec_ramen_model.model"
ramen_model = word2vec.Word2Vec.load(model_path)

In [4]:
reviews_TF_IDF = dataiku.Dataset("reviews_TF_IDF")
df = reviews_TF_IDF.get_dataframe()

In [5]:
df.head()

,store_name,address,ward,score,review_cnt,review,texts_tfidf_sorted_top20,id
0,手打式超多加水麺 ののくら,東京都葛飾区亀有3-11-11マーベラス大協ビル 1F,東京都内,3.98,479,2020年、ラーメン103杯目(^^)本日、朝イチで営業を済ませ、次の打ち合わせまで時間が空...,"['うま味', 'えのき', 'パイン', '町田', '仲見世商店街', '町田駅', '...",ID-000001
1,Homemade Ramen 麦苗,東京都品川区南大井6-11-10,東京都内,3.97,731,2020/08/03(月)5度目の訪問！現在は蜜を避けるために、事前記帳(予約)式になってい...,"['afuri', '恵比寿', '柚子', '中村屋', '利山', '中村', '天然水...",ID-000002
2,麺尊 RAGE,東京都杉並区松庵3-37-22レンツェン松庵 1F,東京都内,3.96,723,5年連続で「ミシュランガイド東京」のビブグルマンに掲載され、「The Tabelog Awa...,"['江戸川橋', '油そば', '中華そば', '江戸川橋駅', '神楽', 'そば', '...",ID-000003
3,らぁ麺や 嶋,東京都渋谷区本町3-41-12,東京都内,3.95,177,今年、西新宿五丁目に新しいラーメン屋が出来た。まだ1年も経っていないだろうにすでに期待度が非...,"['記帳', 'いりこ', '黒毛和牛', 'ウェイティング', '火山', 'ボード', ...",ID-000004
4,宍道湖しじみ中華蕎麦 琥珀,東京都大田区西六郷2-1-3,東京都内,3.95,296,本日は貴重な平日のお休みだった為、普段は中々足を運ぶ事の出来ない蒲田方面へ。11時40分に蒲...,"['黒毛和牛', 'soba', '代々木上原', 'トリュフ', '青森シャモロック', ...",ID-000005


In [6]:
tripAdvisor_path = dataiku.Folder("FXMfF0DU").get_path() + "/TripAdvisor_top_TFIDF_words.txt"
f = open(tripAdvisor_path,'r',encoding="utf-8")

In [7]:
words_ta = []
for i,data in enumerate(f):
    word = data.replace("'",'').replace('[','').replace(']','').replace(' ','').replace('\n','')
    words_ta.append(word)

In [8]:
words_ta

['ロンドン',
 'ハマチ',
 'うなぎ',
 '疑い',
 'メインディッシュ',
 '宝石',
 'すし',
 'お刺身',
 '効率的',
 '黒ごま',
 'リラックス',
 '共有',
 '汁物',
 'サーモン',
 'いたこと',
 '推薦',
 'しみ',
 '前菜',
 '盛り合わせ',
 '高品質']

In [9]:
word_ta_vectors = [ramen_model.wv[w] for w in words_ta]

In [11]:
word_list = []
arr = df['texts_tfidf_sorted_top20'].values
for a in arr:
    l = a.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    word_list.append(l)

In [13]:
word_list.append(words_ta)

In [18]:
i = 0
for l in word_list:
    for w in l:
        i += 1

In [21]:
print("{} words detected before removing duplicates".format(i))

14740 words detected before removing duplicates


In [22]:
uniq_words = list(set(itertools.chain.from_iterable(word_list)))

In [23]:
print("{} words detected before removing duplicates".format(len(uniq_words)))

5435 words detected before removing duplicates


In [26]:
def combine(arr, s): 
    return list(combinations(arr, s)) 

In [27]:
array = [21, 18, 19] 
set = 2
print(combine(array, set))

[(21, 18), (21, 19), (18, 19)]


In [28]:
len(combine(uniq_words, 2))

14766895

In [39]:
for w1, w2 in product(array, repeat=2):
    print(w1, w2)

21 21
21 18
21 19
18 21
18 18
18 19
19 21
19 18
19 19


In [33]:
i = 0
for w1, w2 in product(uniq_words, repeat=2):
    i += 1

In [34]:
i

29539225

In [0]:
#scores = {}
#for word1, word2 in product(uniq_words, repeat=2):
#    # print(word1, word2)
#    scores[(word1, word2)] =  cos_sim(ramen_model.wv[word1], ramen_model.wv[word2])

In [0]:
scores = {}
for touple in tqdm(combine(uniq_words, 2)):
    # print(word1, word2)
    similarity = cos_sim(ramen_model.wv[touple[0]], ramen_model.wv[touple[1]])
    scores[(touple[0], touple[1])] = similarity
    scores[(touple[1], touple[0])] = similarity

 71%|███████   | 10430666/14766895 [04:48<02:19, 31007.74it/s]

In [0]:
scores

In [0]:
avg_avg_scores = []
for word_2 in words_ta:
    avg_scores = []
    for review in df["texts_tfidf_sorted_top20"].values:
        review = review.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
        word_cross_scores = []
        for word_1 in review:
            score = scores[(word_1, word_2)]
            word_cross_scores.append(score)
        avg_scores.append(np.mean(word_cross_scores))
    avg_avg_scores.append(np.mean(avg_scores))

In [0]:
avg_avg_scores = []
for review in df["texts_tfidf_sorted_top20"].values:
    review = review.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    avg_scores = []
    for word_1 in review:
        word_cross_scores = []
        for word_2 in words_ta:
            score = scores[(word_1, word_2)]
            word_cross_scores.append(score)
        avg_scores.append(np.mean(word_cross_scores))
    avg_avg_scores.append(np.mean(avg_scores))

In [0]:
len(avg_avg_scores)

In [0]:
df['similarity_score'] = avg_avg_scores

In [0]:
recommendation = dataiku.Dataset("recommendation")
recommendation.write_with_schema(df)